In [4]:
data_dir = '../data/google'
split = 'train'
CAPTIONS_PER_IMAGE = 5
embeddings_num = CAPTIONS_PER_IMAGE

In [5]:
from nltk.tokenize import RegexpTokenizer
from collections import defaultdict
from miscc.config import cfg

import torch
import torch.utils.data as data
from torch.autograd import Variable
import torchvision.transforms as transforms

import os
import sys
import numpy as np
import pandas as pd
from PIL import Image
import numpy.random as random
if sys.version_info[0] == 2:
    import cPickle as pickle
else:
    import pickle

In [6]:
def load_filenames(data_dir, split):
        filepath = '%s/%s/filenames.pickle' % (data_dir, split)
        if os.path.isfile(filepath):
            with open(filepath, 'rb') as f:
                filenames = pickle.load(f)
            print('Load filenames from: %s (%d)' % (filepath, len(filenames)))
        else:
            filenames = []
        return filenames

In [19]:
# from IPython.display import Image
from PIL import Image
data_dir = '../data/google'
split = 'train' 
filenames = load_filenames(data_dir, split)
fail = 0 
for name in filenames:
    # print(name)
    img_path = '../data/google/images/' + str(name) + '.jpg'
    # print(img_path)
    try:
        img = Image.open(img_path).convert('RGB')

    except IOError: 
        fail += 1
        pass 

Load filenames from: ../data/google/train/filenames.pickle (10025)


In [20]:
fail

219

In [21]:
10025 - 219

9806

In [132]:
def load_captions(data_dir, filenames):
        all_captions = []
        for i in range(len(filenames)):
            cap_path = '%s/text/%s' % (data_dir, filenames[i])
            with open(cap_path, "r") as f:
                captions = f.read().decode('utf8').split('\n')
                cnt = 0
                for cap in captions:
                    if len(cap) == 0:
                        continue
                    cap = cap.replace("\ufffd\ufffd", " ")

                    # picks out sequences of alphanumeric characters as tokens
                    # and drops everything else
                    tokenizer = RegexpTokenizer(r'\w+')
                    tokens = tokenizer.tokenize(cap.lower())
                    #print('tokens', tokens)

                    if len(tokens) == 0:
                        print('cap', cap)
                        continue

                    tokens_new = []
                    for t in tokens:
                        t = t.encode('ascii', 'ignore').decode('ascii')
                        if len(t) > 0:
                            tokens_new.append(t)

                    all_captions.append(tokens_new)
                    cnt += 1
                    if cnt == embeddings_num:
                        break
                if cnt < embeddings_num:
                    print('ERROR: the captions for %s less than %d'
                          % (filenames[i], cnt))
        return all_captions

In [134]:
def build_dictionary(train_captions):
        word_counts = defaultdict(float)
        captions = train_captions
        for sent in captions:
            for word in sent:
                word_counts[word] += 1

        vocab = [w for w in word_counts if word_counts[w] >= 0]

        ixtoword = {}
        ixtoword[0] = '<end>'
        wordtoix = {}
        wordtoix['<end>'] = 0
        ix = 1
        for w in vocab:
            wordtoix[w] = ix
            ixtoword[ix] = w
            ix += 1

        train_captions_new = []
        for t in train_captions:
            rev = []
            for w in t:
                if w in wordtoix:
                    rev.append(wordtoix[w])
            # rev.append(0)  # do not need '<end>' token
            train_captions_new.append(rev)

        return [train_captions_new, ixtoword, wordtoix, len(ixtoword)]

In [ ]:
def load_text_data(data_dir, split):
        filepath = os.path.join(data_dir, 'captions.pickle')
        train_names = load_filenames(data_dir, 'train')

        if not os.path.isfile(filepath):
            train_captions = load_captions(data_dir, train_names)

            train_captions, ixtoword, wordtoix, n_words = \
                build_dictionary(train_captions)
            with open(filepath, 'wb') as f:
                pickle.dump([train_captions, ixtoword, wordtoix], f, protocol=2)
                print('Save to: ', filepath)
        else:
            with open(filepath, 'rb') as f:
                x = pickle.load(f)
                train_captions = x[0]
                ixtoword, wordtoix = x[1], x[2]
                del x
                n_words = len(ixtoword)
                print('Load from: ', filepath)
        if split == 'train':
            # a list of list: each list contains
            # the indices of words in a sentence
            captions = train_captions
            filenames = train_names
        else:  # split=='test'
            captions = test_captions
            filenames = test_names
        return filenames, captions, ixtoword, wordtoix, n_words

In [136]:
load_text_data(data_dir, split)

Load filenames from: ../data/google/train/filenames.pickle (10025)
Load filenames from: ../data/google/test/filenames.pickle (10025)
('Load from: ', '../data/google/captions.pickle')


([u'5',
  u'6',
  u'7',
  u'8',
  u'9',
  u'10',
  u'12',
  u'15',
  u'17',
  u'18',
  u'19',
  u'20',
  u'21',
  u'22',
  u'27',
  u'28',
  u'29',
  u'31',
  u'33',
  u'34',
  u'36',
  u'37',
  u'38',
  u'39',
  u'41',
  u'42',
  u'45',
  u'48',
  u'50',
  u'51',
  u'52',
  u'53',
  u'54',
  u'55',
  u'56',
  u'59',
  u'60',
  u'61',
  u'62',
  u'63',
  u'64',
  u'65',
  u'66',
  u'67',
  u'68',
  u'69',
  u'70',
  u'72',
  u'73',
  u'74',
  u'75',
  u'77',
  u'78',
  u'80',
  u'81',
  u'82',
  u'83',
  u'85',
  u'86',
  u'88',
  u'91',
  u'92',
  u'93',
  u'95',
  u'96',
  u'97',
  u'98',
  u'99',
  u'100',
  u'101',
  u'102',
  u'103',
  u'104',
  u'107',
  u'109',
  u'110',
  u'113',
  u'114',
  u'115',
  u'117',
  u'118',
  u'119',
  u'120',
  u'122',
  u'123',
  u'124',
  u'126',
  u'127',
  u'128',
  u'129',
  u'130',
  u'131',
  u'132',
  u'133',
  u'134',
  u'137',
  u'139',
  u'140',
  u'142',
  u'143',
  u'144',
  u'145',
  u'147',
  u'148',
  u'150',
  u'152',
  u'153',
  u

In [4]:
def load_captions(data_dir, filenames):
        all_captions = []
        for i in range(len(filenames)):
            cap_path = '%s/text/%s.txt' % (data_dir, filenames[i])
            with open(cap_path, "r") as f:
                captions = f.read().decode('utf8').split('\n')
                cnt = 0
                for cap in captions:
                    if len(cap) == 0:
                        continue
                    cap = cap.replace("\ufffd\ufffd", " ")
                    # picks out sequences of alphanumeric characters as tokens
                    # and drops everything else
                    tokenizer = RegexpTokenizer(r'\w+')
                    tokens = tokenizer.tokenize(cap.lower())
                    # print('tokens', tokens)
                    if len(tokens) == 0:
                        print('cap', cap)
                        continue

                    tokens_new = []
                    for t in tokens:
                        t = t.encode('ascii', 'ignore').decode('ascii')
                        if len(t) > 0:
                            tokens_new.append(t)
                    all_captions.append(tokens_new)
                    cnt += 1
                    if cnt == embeddings_num:
                        break
                if cnt < embeddings_num:
                    print('ERROR: the captions for %s less than %d'
                          % (filenames[i], cnt))
        return all_captions

In [5]:
def build_dictionary(train_captions, test_captions):
        word_counts = defaultdict(float)
        captions = train_captions + test_captions
        for sent in captions:
            for word in sent:
                word_counts[word] += 1

        vocab = [w for w in word_counts if word_counts[w] >= 0]

        ixtoword = {}
        ixtoword[0] = '<end>'
        wordtoix = {}
        wordtoix['<end>'] = 0
        ix = 1
        for w in vocab:
            wordtoix[w] = ix
            ixtoword[ix] = w
            ix += 1

        train_captions_new = []
        for t in train_captions:
            rev = []
            for w in t:
                if w in wordtoix:
                    rev.append(wordtoix[w])
            # rev.append(0)  # do not need '<end>' token
            train_captions_new.append(rev)

        test_captions_new = []
        for t in test_captions:
            rev = []
            for w in t:
                if w in wordtoix:
                    rev.append(wordtoix[w])
            # rev.append(0)  # do not need '<end>' token
            test_captions_new.append(rev)

        return [train_captions_new, test_captions_new,
                ixtoword, wordtoix, len(ixtoword)]

In [7]:
def load_text_data(data_dir, split):
        filepath = os.path.join(data_dir, 'captions.pickle')
        train_names = load_filenames(data_dir, 'train')
        test_names = load_filenames(data_dir, 'test')
        if not os.path.isfile(filepath):
            train_captions = load_captions(data_dir, train_names)
            test_captions = load_captions(data_dir, test_names)

            train_captions, test_captions, ixtoword, wordtoix, n_words = \
                build_dictionary(train_captions, test_captions)
            with open(filepath, 'wb') as f:
                pickle.dump([train_captions, test_captions,
                             ixtoword, wordtoix], f, protocol=2)
                print('Save to: ', filepath)
        else:
            with open(filepath, 'rb') as f:
                x = pickle.load(f)
                train_captions, test_captions = x[0], x[1]
                ixtoword, wordtoix = x[2], x[3]
                del x
                n_words = len(ixtoword)
                print('Load from: ', filepath)
        if split == 'train':
            # a list of list: each list contains
            # the indices of words in a sentence
            captions = train_captions
            filenames = train_names
        else:  # split=='test'
            captions = test_captions
            filenames = test_names
        return filenames, captions, ixtoword, wordtoix, n_words

### google dic dimentionn

In [8]:
google_cap_pickle =  load_text_data(data_dir, split)

Load filenames from: ../data/google/train/filenames.pickle (10025)
Load filenames from: ../data/google/test/filenames.pickle (10025)
('Load from: ', '../data/google/captions.pickle')


In [9]:
google_cap_pickle[4]

27297

### load coco dictionary: ixtoword, wordtoix, n_words

In [10]:
coco_data_dir = '../data/coco'
coco_cap_pickle = load_text_data(coco_data_dir, 'train')

Load filenames from: ../data/coco/train/filenames.pickle (82783)
Load filenames from: ../data/coco/test/filenames.pickle (40470)
('Load from: ', '../data/coco/captions.pickle')


In [11]:
coco_cap_pickle[4] #coco n_words

27297

In [12]:
def get_coco_dic(data_dir):
        filepath = os.path.join(data_dir, 'captions.pickle')
        train_names = load_filenames(data_dir, 'train')
        test_names = load_filenames(data_dir, 'test')
        
        with open(filepath, 'rb') as f:
            x = pickle.load(f)
            train_captions, test_captions = x[0], x[1]
            ixtoword, wordtoix = x[2], x[3]
            del x
            n_words = len(ixtoword)
            print('Load from: ', filepath)
            
        return ixtoword, wordtoix, n_words

In [13]:
coco_dic = get_coco_dic(coco_data_dir)

Load filenames from: ../data/coco/train/filenames.pickle (82783)
Load filenames from: ../data/coco/test/filenames.pickle (40470)
('Load from: ', '../data/coco/captions.pickle')


In [14]:
coco_ixtoword = coco_dic[0]
coco_wordtoix = coco_dic[1]

### check words in google and coco, how many do they match?

In [15]:
def get_google_dic(data_dir):
        train_names = load_filenames(data_dir, 'train')
        test_names = load_filenames(data_dir, 'test')
        
        train_captions = load_captions(data_dir, train_names)
        test_captions = load_captions(data_dir, test_names)

        train_captions, test_captions, ixtoword, wordtoix, n_words = \
            build_dictionary(train_captions, test_captions)
    
        n_words = len(ixtoword)
        
        return ixtoword, wordtoix, n_words, 

In [25]:
google_data_dir = '../data/google'
google_dic = get_google_dic(google_data_dir)

Load filenames from: ../data/google/train/filenames.pickle (10025)
Load filenames from: ../data/google/test/filenames.pickle (10025)


In [26]:
google_ixtoword = google_dic[0]
google_wordtoix = google_dic[1]

In [27]:
google_dic[2]

5206

In [19]:
google_word = pd.Series(google_wordtoix.keys())
coco_word = pd.Series(coco_wordtoix.keys())

In [156]:
sum(google_word.isin(coco_word))

4537

In [160]:
len(google_wordtoix)

5206

In [161]:
4537.0 / 5206.0

0.871494429504418

In [28]:
base = 5208 
for word in coco_wordtoix.keys():
    if not google_wordtoix.has_key(word):
        base += 1
        if len(google_wordtoix) >= 27297:
            break
        google_wordtoix[word] = base

In [29]:
len(google_wordtoix)

27297

In [30]:
google_ixtoword = dict((v,k) for k,v in google_wordtoix.iteritems())

In [31]:
len(google_ixtoword)

27297

In [34]:
def modify_google_load_text_data(data_dir, google_ixtoword, google_wordtoix):
        filepath = os.path.join(data_dir, 'captions.pickle')
        train_names = load_filenames(data_dir, 'train')
        test_names = load_filenames(data_dir, 'test')
        if not os.path.isfile(filepath):
            train_captions = load_captions(data_dir, train_names)
            test_captions = load_captions(data_dir, test_names)

            train_captions, test_captions, ixtoword, wordtoix, n_words = \
                build_dictionary(train_captions, test_captions)
            ixtoword = google_ixtoword
            wordtoix = google_wordtoix
            n_words = len(ixtoword)
            with open(filepath, 'wb') as f:
                pickle.dump([train_captions, test_captions,
                             ixtoword, wordtoix], f, protocol=2)
                print('Save to: ', filepath)
        return n_words

In [35]:
google_data_dir = '../data/google'

In [36]:
modify_google_load_text_data(data_dir, google_ixtoword, google_wordtoix)

Load filenames from: ../data/google/train/filenames.pickle (10025)
Load filenames from: ../data/google/test/filenames.pickle (10025)
('Save to: ', '../data/google/captions.pickle')


27297